# Brain Explorer

In [ ]:
import gdown
import os
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from IPython.display import display, clear_output

mcc = MouseConnectivityCache(manifest_file='./.mouse_connectivity/mouse_connectivity_manifest.json', resolution=25)
data_dir = './.mouse_connectivity'
tree_file = f"{data_dir}/tree.pickle"

if not os.path.isfile(tree_file):
    gdown.download("https://drive.google.com/uc?id=1go1rdESSip2TKT1xFKbYjQRFcCN_IoOq", tree_file, quiet=False)

data_file = f'{data_dir}/stats.parquet'
section_data_file = f'{data_dir}/stats-sections.parquet'

clear_output()

%matplotlib inline
from explorer.brain_data_ui import BrainAggregatesHistogramPlot
from explorer.ui import ResultsSelector, RawDataResultsSelector, SectionDataResultsSelector
from figures.clean_data import prepare_data
import ipywidgets as widgets
import pandas as pd


plot_pane = widgets.Output()

def plot_histogram(clean=False, title="Histogram"):
    with plot_pane:
        if not os.path.isfile(data_file):
            gdown.download("https://drive.google.com/uc?id=1EH--7lSNPs7lxTe5H1pEJr7oLOiQNTqm", data_file, quiet=False)

        data = pd.read_parquet(data_file)
        if clean:
            data = prepare_data(data)
        selector = ResultsSelector(data)
        hist_pane = BrainAggregatesHistogramPlot(data_dir, selector, title)
        with plot_pane:
            clear_output()
            display(hist_pane)

def plot_section_histogram(b):
    with plot_pane:
        if not os.path.isfile(section_data_file):
            gdown.download("https://drive.google.com/uc?id=1LAIuXA7xKZ0JDbmzesDxvwCqpCztF98Q", section_data_file, quiet=False)
        clear_output()
        data = pd.read_parquet(section_data_file)
        selector = SectionDataResultsSelector(data)
        hist_pane = BrainAggregatesHistogramPlot(data_dir, selector, "Anterior-Posterior Data Histogram")
        with plot_pane:
            display(hist_pane)

new_hist_button = widgets.Button(description='New histogram', layout=widgets.Layout(width='auto'))
new_hist_button.on_click(lambda b: plot_histogram(False, "Raw Data Histogram"))

new_clean_hist_button = widgets.Button(description='New filtered data histogram', layout=widgets.Layout(width='auto'))
new_clean_hist_button.on_click(lambda b: plot_histogram(True, "Filtered Data Histogram"))

new_ap_hist_button = widgets.Button(description='New anterior-posterior histogram', layout=widgets.Layout(width='auto'))
new_ap_hist_button.on_click(plot_section_histogram)

clear_button = widgets.Button(description='Clear')
clear_button.on_click(lambda b: plot_pane.clear_output())

clean_data = widgets.Checkbox(value=False, description="Clean data")

display(widgets.HBox((new_hist_button, new_clean_hist_button, new_ap_hist_button, clear_button)))
display(plot_pane)